In [14]:
import pandas as pd

data = pd.read_json('../Crawler/webscraper/webscraper/spiders/data.json')

print(data.isnull().sum())
data = data.dropna()
data = data.fillna('')
print(data.isnull().sum())

doc = []
urls = []




data['text'] = data['title'] + ' ' + data['content']
texts = data['text'].tolist()
urls = data['link'].tolist()


print(data[:2])
# print(urls[:1])






title      20
content     0
link        0
dtype: int64
title      0
content    0
link       0
dtype: int64
                  title                                            content  \
0  Association football  , commonly known as , or , is a  played betwee...   
1     Shrewsbury School   is a  (English   for pupils aged 13 –18) in ....   

                                                link  \
0  https://en.wikipedia.org/wiki/Association_foot...   
1    https://en.wikipedia.org/wiki/Shrewsbury_School   

                                                text  
0  Association football , commonly known as , or ...  
1  Shrewsbury School  is a  (English   for pupils...  


title      20
content     0
link        0
text       20
dtype: int64


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

In [16]:
import pickle

# Save the TF-IDF matrix
with open('tfidf_matrix.pkl', 'wb') as file:
    pickle.dump(tfidf_matrix, file)

# Save the vectorizer (for future transformations)
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

# Save the URLs list to keep track of documents
with open('urls.pkl', 'wb') as file:
    pickle.dump(urls, file)


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

# Load the saved objects
with open('tfidf_matrix.pkl', 'rb') as file:
    tfidf_matrix = pickle.load(file)

with open('vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

with open('urls.pkl', 'rb') as file:
    urls = pickle.load(file)

def handle_query(query, top_k=5):
    # Convert the query to a TF-IDF vector
    query_vector = vectorizer.transform([query])

    # Calculate cosine similarity between the query vector and the TF-IDF matrix
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix)[0]

    # Get the indices of the top-K most similar documents
    top_k_indices = similarity_scores.argsort()[-top_k:][::-1]

    # Retrieve the URLs of the top-K most similar documents
    top_k_urls = [urls[i] for i in top_k_indices]

    return top_k_urls

# Example usage
query = 'soccer history'
top_k_urls = handle_query(query, top_k=5)
print('Top 5 most relevant URLs:', top_k_urls)


Top 5 most relevant URLs: ['https://en.wikipedia.org/wiki/English_public_school_football_games', 'https://en.wikipedia.org/wiki/Roman_historiography', 'https://en.wikipedia.org/wiki/Peter_Brown_(historian)', 'https://en.wikipedia.org/wiki/Christopher_Kelly_(historian)', 'https://en.wikipedia.org/wiki/Livy']
